In [44]:
# !pip install python-dotenv

In [42]:
# !pip install langchain-openai

In [43]:
# !pip install langchain

### Poll Generator

In [14]:
import pandas as pd
import os
from dotenv import load_dotenv
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from react_template import get_react_prompt_template
from google.cloud import bigquery
from google.oauth2 import service_account
from langchain.tools.render import render_text_description

In [16]:
# Create a .env file to save yout model api keys
# Specify the path to your .env file if it's not in the same directory as your script
dotenv_path = '.env'

# Load the environment variables from your .env file
load_dotenv(dotenv_path)

api_key = os.getenv('OPENAI_API_KEY')
# api_key = abcd1234
model = ChatOpenAI(model="gpt-4", api_key=api_key)

#### Fetch data fom BigQuery

In [77]:
# client = bigquery.Client(credentials=credentials)

In [78]:
# def get_top_5_content():
#     """Returns a dataframe of content_id and story_headline for the top 3 stories"""
    
#     query = """
#         """
    
#     query_job = client.query(query)
#     results = query_job.result()
    
#     return results.to_dataframe()

In [79]:
# top_5_content = get_top_5_content()

In [69]:
top_content = [{'content_id': '11399681',
  'story_headline': "10 ways world could spiral into nuke war as summer of Putin victories would mark first steps to WW3, army bosses warn. Scroll down to see The Sun's graphic on how Putin's summer offensive could turn into WW3 in 10 steps"},
 {'content_id': '11403108',
  'story_headline': 'Alice Stewart suddenly dead at 58 after CNN correspondent & ex-Ted Cruz adviser’s body found outside near her home. Stewart appeared on CNN as recently as Friday night'},
 {'content_id': '11407921',
  'story_headline': 'Watch moment giant meteor travelling at 1,700mph turns night sky blue over Spain and Portugal in rare spectacle. Watch the epic moment in the above video'},
 {'content_id': '11359687',
  'story_headline': 'Everyone sees the rabbit – but you have high IQ and 20/20 vision if you can find the diamond ring in under 15 seconds. Keep reading to challenge yourself with more puzzling brain teasers'},
 {'content_id': '11409310',
  'story_headline': 'Chilling footage shows Iran’s president Ebrahim Raisi in helicopter moments before crash as fog hampers frantic search. The controversial leader is feared dead as search crews struggle to reach the crash site'}]

In [73]:
# convert top_content to a dataframe
top_5_content = pd.DataFrame(top_content)


In [74]:
top_5_content

,content_id,story_headline
0,11399681,10 ways world could spiral into nuke war as su...
1,11403108,Alice Stewart suddenly dead at 58 after CNN co...
2,11407921,"Watch moment giant meteor travelling at 1,700m..."
3,11359687,Everyone sees the rabbit – but you have high I...
4,11409310,Chilling footage shows Iran’s president Ebrahi...


In [75]:
for story in top_5_content.itertuples():
    print(f"Content ID: {story.content_id}")
    print(f"Story Headline: {story.story_headline}")
    print("\n")

Content ID: 11399681
Story Headline: 10 ways world could spiral into nuke war as summer of Putin victories would mark first steps to WW3, army bosses warn. Scroll down to see The Sun's graphic on how Putin's summer offensive could turn into WW3 in 10 steps


Content ID: 11403108
Story Headline: Alice Stewart suddenly dead at 58 after CNN correspondent & ex-Ted Cruz adviser’s body found outside near her home. Stewart appeared on CNN as recently as Friday night


Content ID: 11407921
Story Headline: Watch moment giant meteor travelling at 1,700mph turns night sky blue over Spain and Portugal in rare spectacle. Watch the epic moment in the above video


Content ID: 11359687
Story Headline: Everyone sees the rabbit – but you have high IQ and 20/20 vision if you can find the diamond ring in under 15 seconds. Keep reading to challenge yourself with more puzzling brain teasers


Content ID: 11409310
Story Headline: Chilling footage shows Iran’s president Ebrahim Raisi in helicopter moments be

#### Create functions that use agents to generate content

In [20]:
def is_headline_appropriate(headline_story):
    prompt_template = ChatPromptTemplate.from_template("You are an expert on generating meaningful and healthy polls for news articles to increase healthy engagement with readers. Only serious news article needs to be approved. Riddles, games, puzzles and brainteasers are not appropriate. Stricly answer in one word, 'Yes' or 'No' if the given headline story is appropriate for poll generation or not: {headline_story}")
    output_parser = StrOutputParser()
    chain = prompt_template | model | output_parser
    result = chain.invoke({"headline_story": headline_story})
    return result

In [21]:
# Define the function to generate poll questions
def generate_poll_question(headline_story):
    prompt_template = PromptTemplate.from_template("You are a good UK citizen and also a poll generating agent that asks good and sincere poll question to the news article readers based on a story headline. Generate a multiple choice poll question for this story headline: {input}?")
    # execute
    output_parser = StrOutputParser()
    chain = prompt_template | model | output_parser
    result = chain.invoke({"input": headline_story})
    return result

In [22]:
# Create a copy to avoid modifying the original dataframe
df = top_5_content.copy() 

# Create a new column 'poll_question'
df['poll_question'] = ''

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
    headline_story = row['story_headline']
    
    # Check if the headline is appropriate
    if is_headline_appropriate(headline_story) == 'Yes':
        # Generate the poll question
        poll_question = generate_poll_question(headline_story)
        df.at[index, 'poll_question'] = poll_question
    else:
        # Leave the 'poll_question' cell empty
        df.at[index, 'poll_question'] = ''


In [23]:
# def append_table_from_dataframe(
#     dataframe: pd.DataFrame,
#     table_id: str,
#     client: bigquery.Client,
#     ) -> None:
#     job_config = bigquery.LoadJobConfig()
#     job_config.create_disposition = "CREATE_IF_NEEDED"
#     job_config.write_disposition = "WRITE_APPEND"
#     job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config)
#     job.result()

In [24]:
# append_table_from_dataframe(df,
#                             table_id = "project.dataset.agent_generated_polls", 
#                             client = client)

In [25]:
# print story  headline and poll question for each story in df

for index, row in df.iterrows():
    print(f"Story Headline: {row['story_headline']}")
    print(f"Poll Question: {row['poll_question']}")
    print("\n")

Story Headline: John F. Kennedy’s son ‘cheated on wife Carolyn Bessette-Kennedy 48 hours before’ they died in plane crash, friend claims. JFK Jr's close friend assumed the president's son had a fling with an old flame after a row with his wife
Poll Question: Based on the recent claims about John F. Kennedy Jr.'s infidelity just before his tragic death, do you believe that the public's perception of him will be significantly affected? 

A) Yes, it tarnishes his image completely.
B) Yes, but it doesn't change my overall opinion of him.
C) No, his personal life doesn't affect my view of him.
D) No, I don't believe the claims. 
E) I am not sure.


Story Headline: Everyone can see the ladder but you have 20/20 vision and a high IQ if you can spot the hidden cat in under 4 seconds. Scroll down for some more mind-boggling fun
Poll Question: 


Story Headline: I was forced to sell my home after HOA took $600 from my paycheck every two weeks over weeds – it was my only way out. See how lawmaker

#### Try Output Parsers

##### Pydantic Output Parser

In [28]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [29]:
# Define your desired data structure.
class Poll(BaseModel):
    poll_question: str = Field(description="Poll question for the news article story")
    option_1: str = Field(description="Option 1 for the poll question")
    option_2: str = Field(description="Option 2 for the poll question")
    option_3: str = Field(description="Option 3 for the poll question")
    option_4: str = Field(description="Option 4 for the poll question")
    option_5: str = Field(description="Option 5 for the poll question")


In [30]:
# Set up a parser + inject instructions into the prompt template.
pydantic_parser = PydanticOutputParser(pydantic_object=Poll)

format_instructions = pydantic_parser.get_format_instructions()

In [31]:
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"poll_question": {"description": "Poll question for the news article story", "title": "Poll Question", "type": "string"}, "option_1": {"description": "Option 1 for the poll question", "title": "Option 1", "type": "string"}, "option_2": {"description": "Option 2 for the poll question", "title": "Option 2", "type": "string"}, "option_3": {"description": "Option 3 for the poll question", "title": "Option 3", "type": "string"}, "option_4": {"description": "Option 4 for the poll question", "title": "Option 4", "type": "string"}, "op

In [32]:
template_string = """You are an AI assistant with extensive knowledge of politics, government systems, and how the world works. Your task is to generate engaging polls for news article readers that promote healthy discussion and critical thinking around political and current event topics.

When generating a poll, keep the following guidelines in mind:

1. Frame the poll question in a neutral and unbiased way, without leading the reader towards a particular answer or political ideology.
2. Provide a range of reasonable answer options that represent different political perspectives and ideologies.
3. Avoid using divisive or emotionally charged language that could inflame tensions or discourage open-minded discussion.
4. Focus on substantive topics that are informative, relevant to the news article or current events, and have real-world political implications.
5. Encourage readers to think critically, consider multiple viewpoints, and form their own opinions based on facts, evidence, and an understanding of how political systems and processes work.
6. If appropriate, include an "Other" option with a free-response field to allow readers to provide their own perspectives.
7. Draw upon your knowledge of political systems, institutions, and processes to provide context and nuance to the poll question and answer options.
8. Aim to engage readers in thoughtful, respectful, and constructive dialogue that fosters a deeper understanding of political issues and encourages civic participation.


 Generate multiple choice poll question with 5 options for this story headline: {input}
 Follow the output formatting instructions: {format_instructions}
"""

In [49]:
prompt_template = PromptTemplate.from_template(template=template_string)

output_parser = StrOutputParser()
chain = prompt_template | model | output_parser
result = chain.invoke({"input": top_5_content['story_headline'][2],"format_instructions": format_instructions})

In [50]:
print('Headline story: ' + top_5_content['story_headline'][2])
print(result)

Headline story: I was forced to sell my home after HOA took $600 from my paycheck every two weeks over weeds – it was my only way out. See how lawmakers hope to crack down on overzealous HOAs
{
  "poll_question": "What is your perspective on the proposed legislative measures for regulating Homeowners Associations (HOAs)?",
  "option_1": "I fully support stronger regulations to protect homeowners from excessive HOA fees and penalties",
  "option_2": "I believe that some regulation is necessary, but homeowners should also fulfill their responsibilities to the community",
  "option_3": "I'm concerned that too much regulation could interfere with the ability of HOAs to maintain community standards and property values",
  "option_4": "I don't think legislative intervention is necessary; disputes should be settled within the community or through legal recourse",
  "option_5": "I have a different perspective or need more information to form an opinion"
}


In [23]:
print(type(result))

<class 'str'>


In [29]:
print(result)

{
"poll_question": "How should financial institutions handle potential fraudulent activity in an account?",
"option_1": "The bank should immediately freeze all transactions and notify the account holder.",
"option_2": "The bank should conduct its own investigation before making any changes to the account.",
"option_3": "The bank should allow transactions to continue, but monitor the account closely for signs of further suspicious activity.",
"option_4": "The bank should immediately return any funds withdrawn due to suspected fraud, even if this could potentially enable fraudulent activity.",
"option_5": "Other (please specify)"
}


### Custom Tool Calling Agent

In [58]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
model = ChatOpenAI(model="gpt-4", api_key=api_key)

@tool
def adding_function(input_1: int, input_2: int) -> int:
    """Adds up the inputs"""
    return input_1 + input_2

tools = [adding_function]

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What will I get if I add ten to ten?"})



> Entering new AgentExecutor chain...

Invoking: `adding_function` with `{'input_1': 10, 'input_2': 10}`


20If you add ten to ten, you will get 20.

> Finished chain.


{'input': 'What will I get if I add ten to ten?',
 'output': 'If you add ten to ten, you will get 20.'}

### Tools

In [52]:
from langchain.pydantic_v1 import BaseModel
from langchain.tools import tool
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "It is going to rain today."

In [53]:
print(search.name)
print(search.description)

search-tool
search-tool(query: str) -> str - Look up things online.


### ReAct

In [54]:
from langchain_core.prompts import PromptTemplate

def get_react_prompt_template():
    # Get the react prompt template
    return PromptTemplate.from_template("""Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
""")

In [55]:
import datetime
from langchain.agents import tool

@tool
def check_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """Returns the current date and time in the specified format"""

    # get the current date and time
    current_time = datetime.datetime.now()
    
    # format the time as a string in the format "YYYY-MM-DD HH:MM:SS"
    formatted_time = current_time.strftime(format)
    
    # return the formatted time
    return formatted_time

In [81]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent

# Setup
llm = ChatOpenAI(model="gpt-4")

# Get a react prompt template
prompt_template = get_react_prompt_template() 

# set the tools
tools = [check_system_time]

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt_template)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Get the current time
agent_executor.invoke({"input": "I need to go on a run at 5. How much time do i have left?"})



> Entering new AgentExecutor chain...
I need to check the current time to determine how much time is left until 5.
Action: check_system_time
Action Input: '%Y-%m-%d %H:%M:%S''2024-05-20 12:26:08'Now that I know the current time, I can calculate the time remaining until 5.
Final Answer: You have approximately 4 hours and 34 minutes left until 5.

> Finished chain.


{'input': 'I need to go on a run at 5. How much time do i have left?',
 'output': 'You have approximately 4 hours and 34 minutes left until 5.'}

### Chat History & Memory

In [82]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

agent_with_chat_history.invoke(
    {"input": "hi! I'm Shachi"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run e35e809f-9821-4148-a11f-a0cd5eb92ea3 not found for run 4aee1da1-610e-4f5f-8ada-94e116423f85. Treating as a root run.




> Entering new AgentExecutor chain...
It seems like Shachi is just introducing herself. There's no question to answer.

Final Answer: Hi Shachi! Nice to meet you.

> Finished chain.


{'input': "hi! I'm Shachi",
 'chat_history': [],
 'output': 'Hi Shachi! Nice to meet you.'}